In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os

# Loading and Cleaning the data

In [2]:
files = "/Users/Tim/Downloads/hotdog-nothotdog/"

In [3]:
train_dir = os.path.join(files, 'train')
validation_dir = os.path.join(files, 'test')

In [18]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

Found 3000 files belonging to 2 classes.
Using 2400 files for training.


In [19]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)

Found 3000 files belonging to 2 classes.
Using 600 files for validation.


In [20]:
class_names = train_ds.class_names

In [21]:
class_names

['hotdog', 'nothotdog']

In [22]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
nomralization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [24]:
normalized_ds = train_ds.map(lambda x, y: (nomralization_layer(x), y))
normalized_val = val_ds.map(lambda x, y: (nomralization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

2023-04-14 14:06:59.775836: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [25]:
num_classes = 2

# Building the model

In [64]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [58]:
#import tensorboard
%load_ext tensorboard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [65]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [67]:
model.fit(
    normalized_ds,
    validation_data=val_ds,
    epochs=50
)

Epoch 1/50
75/75 [==============================] - 3s 37ms/step - loss: 0.7293 - accuracy: 0.5079 - val_loss: 0.9916 - val_accuracy: 0.6683
Epoch 2/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6982 - accuracy: 0.5067 - val_loss: 0.9720 - val_accuracy: 0.6650
Epoch 3/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6952 - accuracy: 0.5158 - val_loss: 0.9733 - val_accuracy: 0.6667
Epoch 4/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6939 - accuracy: 0.5204 - val_loss: 0.9736 - val_accuracy: 0.6667
Epoch 5/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6934 - accuracy: 0.5279 - val_loss: 0.9712 - val_accuracy: 0.6667
Epoch 6/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6931 - accuracy: 0.5279 - val_loss: 0.9703 - val_accuracy: 0.6667
Epoch 7/50
75/75 [==============================] - 3s 37ms/step - loss: 0.6929 - accuracy: 0.5283 - val_loss: 0.9708 - val_accuracy: 0.6667
Epoch 8/50
75

# MobileNet

In [68]:
modelnet = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                             include_top=False,
                                             weights='imagenet')

In [69]:
modelnet.trainable = False

In [75]:
model = tf.keras.Sequential([
    modelnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [76]:
late = .0001

In [78]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
)

Epoch 1/50


2023-04-14 15:21:07.904460: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 [==============================] - ETA: 0s - loss: 0.9456 - accuracy: 0.5263

2023-04-14 15:21:11.172985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 [==============================] - 6s 58ms/step - loss: 0.9456 - accuracy: 0.5263 - val_loss: 0.6869 - val_accuracy: 0.5833
Epoch 2/50
75/75 [==============================] - 3s 35ms/step - loss: 0.9019 - accuracy: 0.5492 - val_loss: 0.6675 - val_accuracy: 0.6117
Epoch 3/50
75/75 [==============================] - 2s 33ms/step - loss: 0.8670 - accuracy: 0.5717 - val_loss: 0.6522 - val_accuracy: 0.6450
Epoch 4/50
75/75 [==============================] - 2s 33ms/step - loss: 0.8376 - accuracy: 0.5929 - val_loss: 0.6399 - val_accuracy: 0.6550
Epoch 5/50
75/75 [==============================] - 3s 39ms/step - loss: 0.8126 - accuracy: 0.6092 - val_loss: 0.6299 - val_accuracy: 0.6650
Epoch 6/50
75/75 [==============================] - 3s 37ms/step - loss: 0.7911 - accuracy: 0.6221 - val_loss: 0.6216 - val_accuracy: 0.6733
Epoch 7/50
75/75 [==============================] - 3s 37ms/step - loss: 0.7723 - accuracy: 0.6346 - val_loss: 0.6147 - val_accuracy: 0.6817
Epoch 8/50
75/75 [======